# Scikit-Learn binary classification model.  Remote train via Azure ML Compute (AML Cluster) 
## and optional HyperDrive (hyper-parameter tuning with multiple child runs) 

This notebook showcases the creation of a ScikitLearn Binary classification model by remotely training on Azure ML Compute Target (AMLCompute Cluster). It shows multiple ways of remote training like using a single Estimator, a ScriptRunConfig and hyper-parameter tunning with HyperDrive with multiple child trainings

## Check library versions
This is important when interacting with different executions between remote compute environments (cluster) and the instance/VM with the Jupyter Notebook.
If not using the same versions you can have issues when creating .pkl files in the cluster and downloading them to load it in the Jupyter notebook.

In [ ]:
# Check versions
import azureml.core
import sklearn
import joblib
import pandas 

print("Azure SDK version:", azureml.core.VERSION)
print('scikit-learn version: {}.'.format(sklearn.__version__))
print('joblib version: {}.'.format(joblib.__version__))
print('pandas version: {}.'.format(pandas.__version__))

## Setup and connect to AML Workspace

In [ ]:
from azureml.core import Workspace

#ws = Workspace.from_config('../../../config.json') # if you run this locally download config.json and place it in root folder of the repo
ws = Workspace.from_config()

print(ws.name, ws.resource_group, ws.location, sep='\n')

## Create An Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
from azureml.core import Experiment

experiment_name = 'amlcompute-train-german-credit'
experiment = Experiment(workspace=ws, name=experiment_name)

## Introduction to Azure ML Compute

Azure ML supports a range of compute targets, which you can define in your workpace and use to run experiments; paying for the resources only when using them.

### Create project directory and copy the training script into the project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on

In [ ]:
import os
import shutil
project_folder = './amlcompute-train-german-credit'
os.makedirs(project_folder, exist_ok=True)

# Copy the training script into the project directory
shutil.copy('train.py', project_folder)

### Connect or Create a Remote AML compute cluster

Try to use the compute target you had created before (make sure you provide the same name here in the variable `cpu_cluster_name`).
If not available, create a new cluster from the code.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Create an AzureMl Compute resource (a container cluster)
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2', 
                                                           vm_priority='dedicated', 
                                                           max_nodes=2)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

### Fetch the AML Dataset

In [ ]:
aml_dataset = ws.datasets['german-credit']

## Create Environment 
#### Optionally list all the available environments and packages in your AML Workspace

In [ ]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)

# List Environments and packages in my workspace
#for env in envs:
    #if env.startswith("AzureML"):
        #print("Name", env)
        #print("packages", envs[env].python.conda_dependencies.serialize_to_string())
        
# Use curated environment from AML named "AzureML-Tutorial"
curated_environment = Environment.get(workspace=ws, name="AzureML-Tutorial")
#print("packages", curated_environment.python.conda_dependencies.serialize_to_string())

# Custom environment: 
# Environment.get(workspace=ws, name="myenv", version="1")

## Configure & Run

## Configure & Run using ScriptRunConfig & Environment 
(Easiest path using curated environments)

In [ ]:
# Add training script to run config
from azureml.core import ScriptRunConfig, RunConfiguration, Experiment

script_runconfig = ScriptRunConfig(source_directory=project_folder, 
                                   script="train.py",
                                   arguments=[aml_dataset.as_named_input('credit')]
                                  )

# Attach compute target to run config
script_runconfig.run_config.target = cpu_cluster
# runconfig.run_config.target = "local"

# Attach environment to run config
script_runconfig.run_config.environment = curated_environment

### Run the experiment with a single ScriptRunConfig and single run (Optional Jump)
Jump this step if you want to use HyperDrive (Go to Option C)

In [ ]:
# Submit the Experiment Run to the AML Compute 
run = experiment.submit(script_runconfig)
run

### Monitor Run 

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

#### Get log results upon completion
Model training and monitoring happen in the background. Wait until the model has finished training before you run more code. Use wait_for_completion to show when the model training is finished:

In [ ]:
run.wait_for_completion(show_output=True)  # specify True for a verbose log

## Configure and Run with Intelligent hyperparameter tuning (HyperDrive using Estimator)

The adjustable parameters that govern the training process are referred to as the hyperparameters of the model. The goal of hyperparameter tuning is to search across various hyperparameter configurations and find the configuration that results in the best performance.

To demonstrate how Azure Machine Learning can help you automate the process of hyperarameter tuning, we will launch multiple runs with different values for numbers in the sequence. First let's define the parameter space using random sampling.

### 1. Create a hyperparameter sweep
First, we will define the hyperparameter space to sweep over. 
In this example we will use random sampling to try different configuration sets of hyperparameters to maximize our primary metric, Accuracy.

In [ ]:
# from azureml.train.hyperdrive import *
from azureml.train.hyperdrive import RandomParameterSampling, BayesianParameterSampling 
from azureml.train.hyperdrive import BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform
    
# solver{'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'}, default='lbfgs'
# penalty{'l1', 'l2', 'elasticnet', 'none'}, default='l2' --- Note that some penalty parameters are not supported by some algorithms..
param_sampling = RandomParameterSampling({
                                          "--solver": choice('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'),
                                          "--penalty": choice('l2')
                                         }
                                        )
# Details on Scikit-Learn LogisticRegression hyper-parameters:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

Now we will define an early termination policy. The BanditPolicy basically states to check the job every 2 iterations. If the primary metric (defined later) falls outside of the top 10% range, Azure ML terminate the job. This saves us from continuing to explore hyperparameters that don't show promise of helping reach our target metric.


In [ ]:
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)
# Note that early termination policy is currently NOT supported with Bayesian sampling
# Check here for recommendations on the multiple policies:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#picking-an-early-termination-policy

Now we are ready to configure a run configuration object, and specify the primary metric 'Accuracy' that's recorded in your training runs. 
If you go back to visit the training script, you will notice that this value is being logged. 
We also want to tell the service that we are looking to maximizing this value. 
We also set the number of samples to 20, and maximal concurrent job to 4.

In [ ]:
# Note that in this case when using HyperDrive, the original Estimator's parameters are not used but the HyperDrive parameters...
hyperdrive_config = HyperDriveConfig(run_config=script_runconfig, 
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=10,
                                     max_concurrent_runs=4)

### 2. Lauch the hyperparameter tuning job.

In [ ]:
# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

# Check here how to submit the hyperdrive run as a step of an AML Pipeline:
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-parameter-tuning-with-hyperdrive.ipynb

### 3. Monitor the HyperDrive run

Monitor the progress of the runs with the Jupyter widget.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

In [ ]:
# Get details for debugging:
RunDetails(hyperdrive_run).get_widget_data()

##### Get log results upon completion
Model training and monitoring happen in the background. Wait until the model has finished training before you run more code. Use wait_for_completion to show when the model training is finished:

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

### 4. Find and get the best model found by HyperDrive¶ 
When all jobs finish, we can find out the one that has the highest accuracy.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
# print(best_run.get_details())

In [ ]:
# Copy 'best_run' to 'run' to re-use the same code also used without HyperDrive
run = best_run

## Display run metrics results
You now have a model trained on a remote cluster. Retrieve the accuracy of the model:

In [ ]:
print(run.get_metrics())

## See files associated with the run

In [ ]:
print(run.get_file_names())

run.download_file('azureml-logs/70_driver_log.txt')

## Download Scikit-Learn model

In [ ]:
# retrieve model for visualization and deployment
# Download the model .pkl file to local (Using the 'run' object)
run.download_file('outputs/model.pkl')

## Register the model
Once you've trained the model, you can save and register it to your workspace. Model registration lets you store and version your models in your workspace to simplify model management and deployment.

Running the following code will register the model to your workspace, and will make it available to reference by name in remote compute contexts or deployment scripts. 

In [ ]:
from azureml.core.model import Model

model_reg = run.register_model(model_name='creditmodel_aml',    # Name of the registered model in your workspace.
                               description='Binary classification model for German credit risk data',
                               model_path='outputs/model.pkl', # Local file to upload and register as a model.
                               model_framework=Model.Framework.SCIKITLEARN,     # Framework used to create the model.
                               model_framework_version='0.20.3',                # Version of scikit-learn used to create the model.
                               tags={'ml-task': "binary-classification", 'business-area': "Risk"},
                               properties={'joblib-version': "0.14.0", 'pandas-version': "0.23.4"},
                               sample_input_dataset=aml_dataset
                              )
model_reg

In [ ]:
# Download Scikit-Learn model pickle file from the model registry
from azureml.core.model import Model
print(Model.get_model_path('creditmodel_aml', _workspace=ws))

model_from_registry = Model(ws,'creditmodel_aml')
model_from_registry.download(target_dir='.', exist_ok=True)